In [1]:
import os
import pickle
import tqdm
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import sklearn
from sklearn.ensemble import BaggingClassifier
from torch.optim import Adam
import warnings
import joblib

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0,
                                     num_parts_to_read: int = 2, columns=None, verbose=False) -> pd.DataFrame:
    res = []
    dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset)
                              if filename.startswith('train')])
    print(dataset_paths)

    start_from = max(0, start_from)
    chunks = dataset_paths[start_from: start_from + num_parts_to_read]
    if verbose:
        print('Reading chunks:\n')
        for chunk in chunks:
            print(chunk)
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        print('chunk_path', chunk_path)
        chunk = pd.read_parquet(chunk_path,columns=columns)
        res.append(chunk)

    return pd.concat(res).reset_index(drop=True)


def create_dataset(path):
    for i, elem in enumerate(['df_'+str(a) for a in range(0, 12)]):
        elem = read_parquet_dataset_from_local(path, i, 1)
        if i==0:
            df = elem.copy()
        else:
            df = pd.concat([df, elem], ignore_index=True, axis=0)
        del elem
    return df


def one_hot_enc(df_1, targets=None):
    cat_columns = ['pre_since_opened', 'pre_since_confirmed', 'pre_pterm',
       'pre_fterm', 'pre_till_pclose', 'pre_till_fclose',
       'pre_loans_credit_limit', 'pre_loans_next_pay_summ',
       'pre_loans_outstanding', 'pre_loans_total_overdue',
       'pre_loans_max_overdue_sum', 'pre_loans_credit_cost_rate', 'pre_loans5',
       'pre_loans530', 'pre_loans3060', 'pre_loans6090', 'pre_loans90', 
        'pre_util', 'pre_over2limit', 'pre_maxover2limit', 'enc_paym_0', 'enc_paym_1', 'enc_paym_2',
       'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7',
       'enc_paym_8', 'enc_paym_9', 'enc_paym_10', 'enc_paym_11', 'enc_paym_12',
       'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16',
       'enc_paym_17', 'enc_paym_18', 'enc_paym_19', 'enc_paym_20',
       'enc_paym_21', 'enc_paym_22', 'enc_paym_23', 'enc_paym_24',
       'enc_loans_account_holder_type', 'enc_loans_credit_status',
       'enc_loans_credit_type', 'enc_loans_account_cur']

    ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore', dtype='int8')
    df_1[ohe.get_feature_names_out()] = ohe.fit_transform(df_1[cat_columns])
    df_1.drop(columns=cat_columns, inplace=True)
    return df_1


def group_df(df_1, targets=None):
    def new_features(df):
        df_1 = df.copy()
        df_1 = df_1.iloc[:, :2].copy().groupby(['id']).max()
        df_1.columns = ['max_rn']
        return df_1
    
    df_max_rn = new_features(df_1)
    df_1 = df_1.groupby(['id']).sum()
    return df_1.merge(df_max_rn, how='left', on='id').drop(columns='rn')


def main():
    path = 'train_data/'
    df = create_dataset(path)
    targets = pd.read_csv('train_target.csv')

    preprocessor = Pipeline(steps=[
        ('encoder', FunctionTransformer(one_hot_enc)),
        ('group_dataset', FunctionTransformer(group_df))
         ])

    df =  preprocessor.fit_transform(df, targets)
    df.loc[:, 'target'] = targets.loc[:, 'flag']

    df.drop_duplicates(inplace=True)
    int_64_cols  = df.select_dtypes('int64').columns
    df[int_64_cols] = df[int_64_cols].astype('int8')
    
    X, y =  df[df.columns[:-1]].to_numpy(), df[df.columns[-1]]

    model = xgboost.XGBClassifier(
    n_estimators=340,
    objective='binary:logistic',
    max_depth=10,
    learning_rate=0.047,
    subsample=0.9268,
    colsample_bytree=0.7107,
    colsample_bylevel=0.8465,
    colsample_bynode=0.4592,
    eval_metric='auc').fit(X, y, verbose=False)

    predict_xgb = model.predict(X)
    print('roc_auc: ', roc_auc_score(y, predict_xgb))
    joblib.dump(meta_classifier, 'ML_project.joblib')

if __name__ == '__main__':
    main()

['train_data/train_data_0.pq', 'train_data/train_data_1.pq', 'train_data/train_data_10.pq', 'train_data/train_data_11.pq', 'train_data/train_data_2.pq', 'train_data/train_data_3.pq', 'train_data/train_data_4.pq', 'train_data/train_data_5.pq', 'train_data/train_data_6.pq', 'train_data/train_data_7.pq', 'train_data/train_data_8.pq', 'train_data/train_data_9.pq']


C:\Users\bogho\AppData\Local\Temp\ipykernel_1152\1573943305.py:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

chunk_path train_data/train_data_0.pq
['train_data/train_data_0.pq', 'train_data/train_data_1.pq', 'train_data/train_data_10.pq', 'train_data/train_data_11.pq', 'train_data/train_data_2.pq', 'train_data/train_data_3.pq', 'train_data/train_data_4.pq', 'train_data/train_data_5.pq', 'train_data/train_data_6.pq', 'train_data/train_data_7.pq', 'train_data/train_data_8.pq', 'train_data/train_data_9.pq']


C:\Users\bogho\AppData\Local\Temp\ipykernel_1152\1573943305.py:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):


...

Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

chunk_path train_data/train_data_8.pq
['train_data/train_data_0.pq', 'train_data/train_data_1.pq', 'train_data/train_data_10.pq', 'train_data/train_data_11.pq', 'train_data/train_data_2.pq', 'train_data/train_data_3.pq', 'train_data/train_data_4.pq', 'train_data/train_data_5.pq', 'train_data/train_data_6.pq', 'train_data/train_data_7.pq', 'train_data/train_data_8.pq', 'train_data/train_data_9.pq']


C:\Users\bogho\AppData\Local\Temp\ipykernel_1152\1573943305.py:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

chunk_path train_data/train_data_9.pq
roc_auc:  0.7742649296396209
